In [34]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords # words which doesn't add value to data
from nltk.stem.porter import PorterStemmer # gives the root word of the letter
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split,StratifiedKFold,GridSearchCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dai\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# Printing stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

* Data Pre-processing

In [5]:
news = pd.read_csv(r"C:\Users\dai\Desktop\A-AI\mini projects\FakeNews_ML\fake-news\train.csv",index_col=0)

In [6]:
news.shape

(20800, 4)

In [7]:
news.head()

,title,author,text,label
id,,,,
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
news.isnull().sum()

title      558
author    1957
text        39
label        0
dtype: int64

In [9]:
news.fillna(' ',inplace = True)

In [10]:
news.isnull().sum().sum()

0

In [11]:
news['content'] = news['author']+' '+news['title']

In [12]:
news['content']

id
0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object

In [13]:
port_stem = PorterStemmer()

In [14]:
def stemming(content):
    stem_content = re.sub('[^a-zA-Z]',' ',content)# ^ EXCLUDE  
    stem_content = stem_content.lower()
    stem_content = stem_content.split()
    stem_content = [port_stem.stem(word) for word in stem_content if not word in stopwords.words('english')]
    stem_content = ' '.join(stem_content)
    return stem_content
    

In [15]:
news['content'] = news['content'].apply(stemming)

In [16]:
print(news['content'])

id
0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [17]:
X = news['content'].values
y = news['label'].values

In [18]:
print(X,y)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv'] [1 0 1 ... 0 1 1]


In [19]:
y.shape

(20800,)

In [20]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [21]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

In [22]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, random_state=24, test_size= 0.3, stratify=y)

In [36]:
lr = LogisticRegression()
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)

In [38]:
accuracy_score(y_test,y_pred)

0.9738782051282051

In [42]:
rf = RandomForestClassifier(random_state=24)

In [44]:
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9931089743589744

In [30]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=24)
params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['log2']
}
rf = RandomForestRegressor(random_state=24)
gcv = GridSearchCV(rf, param_grid=params, cv=kfold, scoring='r2',verbose=3)
gcv.fit(X, y)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV 1/5] END max_depth=10, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=0.040 total time=   1.1s
[CV 2/5] END max_depth=10, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=0.047 total time=   1.1s
[CV 3/5] END max_depth=10, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=0.042 total time=   1.0s
[CV 4/5] END max_depth=10, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=0.027 total time=   1.0s
[CV 5/5] END max_depth=10, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=0.033 total time=   1.0s
[CV 1/5] END max_depth=10, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=0.040 total time=   2.2s
[CV 2/5] END max_depth=10, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=0.04

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=RandomForestRegressor(random_state=24),
             param_grid={'max_depth': [10, 20], 'max_features': ['log2'],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5],
                         'n_estimators': [100, 200, 300]},
             scoring='r2', verbose=3)

In [46]:
print(gcv.best_params_)
print(gcv.best_score_)

{'max_depth': 20, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
0.0839431191896991


In [48]:
best_m = gcv.best_estimator_
df_imp = pd.DataFrame({'feature':list(X.columns),
              'importance':best_m.feature_importances_ })
df_imp.sort_values('importance', inplace=True)
plt.figure(figsize=(15,15))
plt.barh(df_imp['feature'], df_imp['importance'])
plt.title("Feature Importances Plot for Random Forest")
plt.show()

AttributeError: 'csr_matrix' object has no attribute 'columns'

In [ ]:
X_test = pd.read_csv(r"C:\Users\dai\Desktop\A-AI\mini projects\FakeNews_ML\fake-news\test.csv",index_col=0)

In [ ]:
X_test.head()

In [ ]:
X_new = X_test[0]
prediction = model.predict(X_new)
print(prediction)

if (production[0] == 0):
    print('The news is Real')
else:
    print('The news is Fake')
    

In [ ]:
print(y_test[0])